## 토픽 모델링(Topic Modeling) - 20 뉴스그룹
- 토픽 모델링(Topic Modeling)이란 문서 집합에 숨어 있는 주제를 찾아내는 것입니다.
- 많은 양의 문서를 다 읽고 핵심 주제를 찾는 것은 매우 많은 시간이 소모됩니다.
- 머신러닝 기반의 토픽 모델링에 자주 사용되는 기법은 `LSA`(`Latent Semantic Analysis`)와 `LDA(Latent Dirichlet Allocation)`입니다. 



### 데이터 수집
- `fetch_20newsgroups` API를 활용하여 데이터를 수집합니다. 

In [2]:
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

# 모토사이클, 야구, 그래픽스, 윈도우즈, 중동, 기독교, 전자공학, 의학 8개 주제 추출
categories = ['rec.motorcycles', 'rec.sport.baseball', 'comp.graphics', 'comp.windows.x', 
              'talk.politics.mideast', 'soc.religion.christian', 'sci.electronics', 'sci.med']

# 위 `categories` 변수로 기재된 카테고리만 추출. 
news_df = fetch_20newsgroups(subset='all', remove=('headers', 'footers', 'quotes'), categories = categories, random_state = 0)

# LDA는 `Count`기반의 벡터화만 적용
count_vect = CountVectorizer(max_df = 0.95, max_features = 1000, min_df=2, stop_words='english', ngram_range=(1, 2))
feat_vect = count_vect.fit_transform(news_df.data)
print('CountVectorizer Shape:', feat_vect.shape)

CountVectorizer Shape: (7862, 1000)


- 7862개의 문서가 1000개의 피처로 구성된 행렬 데이터라고 보면 된다. 


### 데이터 모델링
- 이렇게 피처 벡터화된 데이터 세트를 기반으로 `LDA` 토픽 모델링을 수행한다. 
- 토픽의 개수는 8개로 동일하게 맞추도록 조정한다. 
- 모델링 시간 측정을 위해 함수를 작성한다. 

In [3]:
import time
import datetime
def bench_mark(start):
  sec = time.time() - start
  times = str(datetime.timedelta(seconds=sec)).split(".")
  times = times[0]
  print(times)

- `n_components`는 개별 토픽별로 각 `word` 피처가 얼마나 많이 그 토픽에 할당됐는지에 대한 수치를 가지고 있다. 
- 높은 값일수록 해당 `word` 피처는 그 토픽의 중심 `word`가 된다. 


In [4]:
start = time.time()

lda = LatentDirichletAllocation(n_components=8, random_state=0)
lda.fit(feat_vect)

bench_mark(start)

0:00:09


- `components_`의 형태와 속성값을 확인한다. 

In [5]:
print(lda.components_.shape)
lda.components_

(8, 1000)


array([[3.60992018e+01, 1.35626798e+02, 2.15751867e+01, ...,
        3.02911688e+01, 8.66830093e+01, 6.79285199e+01],
       [1.25199920e-01, 1.44401815e+01, 1.25045596e-01, ...,
        1.81506995e+02, 1.25097844e-01, 9.39593286e+01],
       [3.34762663e+02, 1.25176265e-01, 1.46743299e+02, ...,
        1.25105772e-01, 3.63689741e+01, 1.25025218e-01],
       ...,
       [3.60204965e+01, 2.08640688e+01, 4.29606813e+00, ...,
        1.45056650e+01, 8.33854413e+00, 1.55690009e+01],
       [1.25128711e-01, 1.25247756e-01, 1.25005143e-01, ...,
        9.17278769e+01, 1.25177668e-01, 3.74575887e+01],
       [5.49258690e+01, 4.47009532e+00, 9.88524814e+00, ...,
        4.87048440e+01, 1.25034678e-01, 1.25074632e-01]])

- array[8, 1000]으로 구성되어 있는데, 8개의 토픽별로 1000개의 word 피처가 해당 토픽별로 연관도 값을 가지고 있는지 확인하는 코드가 됩니다. 
- 조금더 구체화하기 위해 함수를 구현합니다. 

In [6]:
def display_topics(model, feature_names, no_top_words):
    for topic_index, topic in enumerate(model.components_):
        print('Topic #',topic_index)

        # components_ array에서 가장 값이 큰 순으로 정렬했을 때, 그 값의 array index를 반환. 
        topic_word_indexes = topic.argsort()[::-1]
        top_indexes=topic_word_indexes[:no_top_words]
        
        # top_indexes대상인 index별로 feature_names에 해당하는 word feature 추출 후 join으로 concat
        feature_concat = ' '.join([feature_names[i] for i in top_indexes])                
        print(feature_concat)

- CountVectorizer 객체내의 전체 `word`들의 명칭을 get_features_names()를 통해 추출. 
- 토픽별 가장 연관도가 높은 `word` 15개만 추출하도록 합니다. 

In [8]:
feature_names = count_vect.get_feature_names_out()
display_topics(lda, feature_names, 15)

Topic # 0
year 10 game medical health team 12 20 disease cancer 1993 games years patients good
Topic # 1
don just like know people said think time ve didn right going say ll way
Topic # 2
image file jpeg program gif images output format files color entry 00 use bit 03
Topic # 3
like know don think use does just good time book read information people used post
Topic # 4
armenian israel armenians jews turkish people israeli jewish government war dos dos turkey arab armenia 000
Topic # 5
edu com available graphics ftp data pub motif mail widget software mit information version sun
Topic # 6
god people jesus church believe christ does christian say think christians bible faith sin life
Topic # 7
use dos thanks windows using window does display help like problem server need know run


- 위 결과의 해석은 각자 판단하여 유추할 수 있을 것이다. 몇가지 예를 들면 아래와 같다. 
  + Topic #0: 의학과 관련된 주제가 주를 이루고 있음을 알 수 있다. 
  + Topic #1: 일반적인 단어만 추출이 되어 명확하게 기록되지 않고 있다. 
  + Topic #2: 그래픽스와 관련된 단어가 주로 추출되는 것을 확인할 수 있다. 
  + Topic #6: 기독교 색채가 강한 것을 알 수 있다. 
  + Topic #7: 윈도우 운영제체와 같은 글이 보이는 것을 확인할 수 있다. 
